In [22]:
import h5py
import os
import glob
from keras.models import load_model

models = glob.glob(os.path.join('..', 'Models', '*.h5'))
print(models[0])
filename = models[0]
# def read_hdf5(path):

#     weights = {}

#     keys = []
#     with h5py.File(path, 'r') as f: # open file
#         f.visit(keys.append) # append all keys to list
#         for key in keys:
#             if ':' in key: # contains data if ':' in key
#                 print(f[key].name)
#                 print(dir(f[key]))
#                 # weights[f[key].name] = f[key].value
#     return weights

..\Models\pcc_model_1.h5


In [23]:
cnn_model = load_model(filename)
cnn_model.load_weights

<bound method Model.load_weights of <keras.engine.sequential.Sequential object at 0x000002AB88F22850>>